Работу выполнили: Энзе Гонг и Катричева Надежда

## 1. Подготовка данных

In [1]:
import pandas as pd
import csv

Open train, change NaN for 0.

In [2]:
train = pd.read_csv('train.csv',sep = '\t', quoting=csv.QUOTE_NONE,encoding='utf-8').fillna(0)
len(train)

16406

In [3]:
train.head()

,text,class,cV,cR1,cR2,V,R1,R2
0,"Будучи в прошлый четверг в Софии, он назвал се...",0,0,0,0,0,0,0
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134
2,"Заместитель Генерального секретаря подчеркнул,...",0,0,0,0,0,0,0
3,Продажа недвижимости из собственных портфелей ...,0,0,0,0,0,0,0
4,"Новым является то, что повышенное давление кон...",0,0,0,0,0,0,0


The class-named column hinders using some pandas methods, so let's change the column name.

In [13]:
train.rename(index=str, columns={"class": "class_"}, inplace=True,) # inplace=True means the df is really changed
train.head()

,text,class_,cV,cR1,cR2,V,R1,R2
0,"Будучи в прошлый четверг в Софии, он назвал се...",0,0,0,0,0,0,0
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134
2,"Заместитель Генерального секретаря подчеркнул,...",0,0,0,0,0,0,0
3,Продажа недвижимости из собственных портфелей ...,0,0,0,0,0,0,0
4,"Новым является то, что повышенное давление кон...",0,0,0,0,0,0,0


Classes 0 and 1 in train.csv

In [7]:
train.query('class_ == "0"').class_.count() #same (train.class_ == 0).sum()

10864

In [8]:
train.query('class_ == "1"').class_.count() #same (train.class_ == 1).sum()

5542

Classes 0 and 1 in dev.csv

In [10]:
dev = pd.read_csv('dev.csv',sep = '\t', quoting=csv.QUOTE_NONE).fillna(0)

dev.rename(index=str, columns={"class": "class_"}, inplace=True,)
#dev.head()
(dev.class_ == 0).sum()

2760

In [11]:
(dev.class_ == 1).sum()

1382

Classes 0 and 1 in test.csv

In [12]:
test = pd.read_csv('test.csv',sep = '\t', quoting=csv.QUOTE_NONE).fillna(0)

test.rename(index=str, columns={"class": "class_"}, inplace=True,)
#test.head()
(test.class_ == 0).sum()

1365

In [95]:
(test.class_ == 1).sum()

680

## 2. Анализ разметки

In [14]:
"""We could view only class 1, but it's not necessary"""
#markup = train.loc[train['class_'] == 1]
#markup.head()

"""Drop what's not the ellipsis offset markup cells""" 
stat_train = train.drop(['text', 'class_'], axis=1)

stat_train.head()

,cV,cR1,cR2,V,R1,R2
0,0,0,0,0,0,0
1,92:99,83:91,103:109,127:127,119:124,127:134
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [15]:
"""Change offsets for the number of offsets in a cess"""

labels = ['cV', 'cR1', 'cR2', 'V', 'R1', 'R2']

for i in stat_train.index:
    for label in labels:
        if stat_train.loc[i, label] != 0:
            stat_train.loc[i, label] = len((stat_train.loc[i, label]).split()) # offsets are split by spaces 

stat_train.head()

,cV,cR1,cR2,V,R1,R2
0,0,0,0,0,0,0
1,1,1,1,1,1,1
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [16]:
"""To see the unique rows"""
stat_train.drop_duplicates()

,cV,cR1,cR2,V,R1,R2
0,0,0,0,0,0,0
1,1,1,1,1,1,1
14,1,1,0,1,1,0
117,1,1,1,2,2,2
211,1,1,1,3,3,3
2285,1,1,1,4,4,4
5902,1,1,1,5,5,5
8104,1,0,1,1,1,1


We can see that there are 8 different configurations in test.


To see the numbers of sentences of such configurations we'll use groupby, numbers on the right.

In [25]:
stat_train.groupby(stat_train.columns.tolist(),as_index=False).size()

cV  cR1  cR2  V  R1  R2
0   0    0    0  0   0     10864
1   0    1    1  1   1         1
    1    0    1  1   0        77
         1    1  1   1      5095
              2  2   2       315
              3  3   3        45
              4  4   4         7
              5  5   5         2
dtype: int64

Do the same to the other datasets.


Dev

In [33]:
stat_dev = dev.drop(['text', 'class_'], axis=1)

labels = ['cV', 'cR1', 'cR2', 'V', 'R1', 'R2']

for i in stat_dev.index:
    for label in labels:
        if stat_dev.loc[i, label] != 0:
            stat_dev.loc[i, label] = len((stat_dev.loc[i, label]).split())

stat_dev.drop_duplicates()

,cV,cR1,cR2,V,R1,R2
0,0,0,0,0,0,0
1,1,1,1,1,1,1
61,1,1,1,2,2,2
297,1,1,0,1,1,0
355,1,1,1,3,3,3
888,1,1,1,7,7,7
938,1,1,1,4,4,4


7 different cases in dev, with such ststistics:

In [26]:
stat_dev.groupby(stat_dev.columns.tolist(),as_index=False).size()

cV  cR1  cR2  V  R1  R2
0   0    0    0  0   0     2760
1   1    0    1  1   0       27
         1    1  1   1     1265
              2  2   2       73
              3  3   3        9
              4  4   4        7
              7  7   7        1
dtype: int64

Test

In [31]:
stat_test = test.drop(['text', 'class_'], axis=1)

labels = ['cV', 'cR1', 'cR2', 'V', 'R1', 'R2']

for i in stat_test.index:
    for label in labels:
        if stat_test.loc[i, label] != 0:
            stat_test.loc[i, label] = len((stat_test.loc[i, label]).split())

stat_test.drop_duplicates()

,cV,cR1,cR2,V,R1,R2
0,0,0,0,0,0,0
1,1,1,1,1,1,1
23,1,1,1,2,2,2
254,1,1,0,1,1,0
600,1,1,1,5,5,5
758,1,1,1,3,3,3
1115,1,1,1,4,4,4
1696,1,1,0,2,2,0


8 different cases in dev, with such ststistics:

In [30]:
stat_test.groupby(stat_test.columns.tolist(),as_index=False).size()

cV  cR1  cR2  V  R1  R2
0   0    0    0  0   0     1365
1   1    0    1  1   0       16
              2  2   0        1
         1    1  1   1      616
              2  2   2       42
              3  3   3        3
              4  4   4        1
              5  5   5        1
dtype: int64

# 3. Конвертер разметки-1

In [54]:
class_1 = train.loc[train['class_'] == 1] #The rows with gapping

In [53]:
class_1.head()

,text,class_,cV,cR1,cR2,V,R1,R2
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134
5,"Выворачивает наизнанку, запад превращает в юг,...",1,30:40,24:29,41:45,57:57,49:54,57:65
7,Текущие обязательства компании в 2009 г. сокра...,1,109:120,82:108,121:148,180:180,152:177,180:202
8,Первая конвенция была ратифицирована 80 процен...,1,22:36,0:16,37:74,87:87,78:84,87:116
14,"Но Христианин сказал Упрямому: — Нет, сосед, л...",1,105:112,113:139,0,156:156,156:169,0


In [50]:
def str_to_inx(inx_str):
    '''
    Change slices from strings into integers.
    '''
    inx_str = str(inx_str)
    inx_str = inx_str.split(' ')
    inx_list = []
    for i in inx_str:
        i = i.split(':')
        for inx in i:
            inx_list.append(inx)
    
    return inx_list
        

def IOB_encoder(df):
    '''
    Split texts into words, and map them with their IOB properties.
    '''
    encoded = []
    for i in range(len(df)):
        text = df['text'].iloc[i]
        cv_inx = str_to_inx(df['cV'].iloc[i])
        cR1_inx = str_to_inx(df['cR1'].iloc[i])
        cR2_inx = str_to_inx(df['cR2'].iloc[i])
        V_inx = str_to_inx(df['V'].iloc[i])
        R1_inx = str_to_inx(df['R1'].iloc[i])
        R2_inx = str_to_inx(df['R2'].iloc[i])
        
        cv = [text[int(cv_inx[int(2*n)]):int(cv_inx[int(2*n+1)])] for n in range(int(len(cv_inx)/2))]
        #print(cv)
        cR1 = [text[int(cR1_inx[int(2*n)]):int(cR1_inx[int(2*n+1)])] for n in range(int(len(cR1_inx)/2))]
        cR2 = [text[int(cR2_inx[int(2*n)]):int(cR2_inx[int(2*n+1)])] for n in range(int(len(cR2_inx)/2))]
        V = [text[int(V_inx[int(2*n)]):int(V_inx[int(2*n+1)])] for n in range(int(len(V_inx)/2))]
        R1 = [text[int(R1_inx[int(2*n)]):int(R1_inx[int(2*n+1)])] for n in range(int(len(R1_inx)/2))]
        R2 = [text[int(R2_inx[int(2*n)]):int(R2_inx[int(2*n+1)])] for n in range(int(len(R2_inx)/2))]
        
        IOB = {}
        
        if len(cv) > 0:
            for k in cv:
                text = text.replace(k,'')
                IOB[k] = 'B-cV'
        if len(cR1) > 0:
            for k in cR1:
                text = text.replace(k,'')
                IOB[k] = 'B-cR1'
        if len(cR2) > 0:
            for k in cR2:
                text = text.replace(k,'')
                words = cR2[0].split(' ')
                if len(words) > 1:
                    IOB[words[0]] = 'B-cR2'
                    IOB[' '.join(words[1:])] = 'I-cR2'
                elif len(words) == 1:
                    IOB[words[0]] = 'B-cR2'
                
        if len(V) > 0:
            for k in V:
                text = text.replace(k,'')
                IOB[k] = 'V'
        if len(R1) > 0:
            for k in R1:
                text = text.replace(k,'')
                IOB[k] = 'B-R1'
        if len(R2) > 0:
            for j in range(len(R2)):
                text = text.replace(R2[j],'')
                words = R2[j].split(' ')
                
                if len(words) > 1:
                    IOB[words[0]] = 'B-R2'
                    IOB[' '.join(words[1:])] = 'I-R2'
                else:
                    IOB[words[0]] = 'B-R2'
        
        text = text.split(' ')
        
        for word in text:
            if word not in IOB:
                IOB[word] = 'O'
            
        
        encoded.append(IOB)
        
        
    
    return encoded
    
        
def IOB_decoder(encoded):
    '''
    Turn IOB into slices. Since V class is non-existent, the exact slices generated my differ from the original ones. 
    '''
    decoded = []
    
    for i in range(len(encoded)):
        text = class_1['text'].iloc[i]
        IOB = {}
        for word in encoded[i]:
            if encoded[i][word] == 'B-cV':
                cV_start = text.index(word)
                cV_end = cV_start + len(word)
                IOB['cV'] = str(cV_start) + ':' + str(cV_end)
            elif encoded[i][word] == 'B-cR1':
                cR1_start = text.index(word)
                cR1_end = cV_start + len(word)
                IOB['cR1'] = str(cR1_start) + ':' + str(cR1_end)
            elif encoded[i][word] == 'V':
                V_start = text.index(word)
                V_end = V_start + len(word)
                IOB['V'] = str(V_start) + ':' + str(V_end)
            elif encoded[i][word] == 'B-R1':
                R1_start = text.index(word)
                R1_end = R1_start + len(word)
                IOB['R1'] = str(R1_start) + ':' + str(R1_end)
            elif encoded[i][word] == 'B-cR2':
                cR2_start = text.index(word)
                if 'I-cR2' in encoded[i].values():
                    for word2 in encoded[i]:
                        if encoded[i][word2] == 'I-cR2':
                            cR2_end = text.index(word2) + len(word2)
                    IOB['cR2'] = str(cR2_start) + ':' + str(cR2_end)
                else:
                    cR2_end = text.index(word) + len(word)
                    IOB['cR2'] = str(cR2_start) + ':' + str(cR2_end)
            elif encoded[i][word] == 'B-R2':
                R2_start = text.index(word)
                if 'I-R2' in encoded[i].values():
                    for word2 in encoded[i]:
                        if encoded[i][word2] == 'I-R2':
                            R2_end = text.index(word2) + len(word2)
                    IOB['R2'] = str(R2_start) + ':' + str(R2_end)
                else:
                    R2_end = text.index(word) + len(word)
                    IOB['R2'] = str(R2_start) + ':' + str(R2_end)
        
        decoded.append(IOB)
        
    return decoded

In [51]:
encoded = IOB_encoder(class_1)

In [57]:
encoded[0]

{'состоит': 'B-cV',
 'часть их': 'B-cR1',
 'одного': 'B-cR2',
 '': 'V',
 'часть': 'B-R1',
 'из': 'B-R2',
 'двух': 'I-R2',
 'Работа': 'O',
 'с': 'O',
 'двухбайтовыми': 'O',
 'наборами': 'O',
 'символов': 'O',
 '—': 'O',
 'просто': 'O',
 'кошмар': 'O',
 'для': 'O',
 'программиста,': 'O',
 'так': 'O',
 'как': 'O',
 'байта,': 'O',
 'а': 'O',
 '.': 'O'}

In [47]:
decoded = IOB_decoder(encoded)

In [58]:
decoded[0]

{'cV': '92:99',
 'cR1': '83:100',
 'cR2': '103:109',
 'V': '0:0',
 'R1': '83:88',
 'R2': '100:13'}

## 4. Конвертер разметки-2

# 5. Бинарная классификация

In [60]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from nltk import word_tokenize

Using TensorFlow backend.


In [62]:
def clear_text(df, MAX_length):
    '''
    Decapitalize the text and remove punctuations, numbers, etc.. Also remove sentences that are too long.
    '''
    docs = []
    labels = []
    IOB = []
    text_index = []
    for i in range(len(df)):
        current_pair = []
        sent = df['text'][i]
        sent = sent.lower()
        ru = ' йцукенгшщзхъфывапролджэячсмитьбюё'
        sent = ''.join([letter for letter in sent if letter in ru])
        if len(sent.split(' ')) <= MAX_length:
            docs.append(sent)
            labels.append(df['class_'][i])
            text_index.append(i)
    labels = array(labels)
    return docs, labels, text_index

docs_train, labels_train, text_index = clear_text(train, 50)

In [63]:
def make_vocab(docs):
    '''
    Make a vocabulary for later use.
    '''
    vocab = []
    for i in docs:
        i = i.split(' ')
        for word in i:
            if word not in vocab:
                vocab.append(word)
    
    return vocab

vocab = make_vocab(docs_train)

In [64]:
len(vocab)

63616

In [65]:
docs_train[0:2]

['будучи в прошлый четверг в софии он назвал себя наполовину восточноевропейцем и за последние пять дней успел принять у себя лидеров украины польши и чехии',
 'работа с двухбайтовыми наборами символов  просто кошмар для программиста так как часть их состоит из одного байта а часть  из двух']

In [66]:
vocab_size = len(vocab)
encoded_docs = [one_hot(d, vocab_size) for d in docs_train]#turn sentences from strings into arrays of indexes.

In [68]:
encoded_docs[0:5]

[[49886,
  60558,
  17844,
  56856,
  60558,
  60552,
  56598,
  42147,
  61513,
  8200,
  44424,
  9452,
  43860,
  15979,
  23458,
  41756,
  15782,
  29215,
  52627,
  61513,
  30956,
  3555,
  23001,
  9452,
  54317],
 [35387,
  48962,
  25945,
  48765,
  53016,
  57747,
  61822,
  23224,
  41010,
  57293,
  30894,
  49067,
  32921,
  55973,
  60613,
  56847,
  27042,
  57101,
  49067,
  60613,
  7997],
 [33806,
  61119,
  46142,
  33223,
  59240,
  18080,
  28248,
  45972,
  53989,
  53448,
  26939,
  60558,
  9866,
  16155,
  9452,
  31533,
  32959,
  12434,
  58800,
  9452,
  55040,
  38005,
  40713,
  25850,
  9452,
  2086,
  35600,
  32877,
  58537,
  19891,
  27674,
  61629,
  48459,
  14013,
  39063,
  41930,
  9452,
  56860],
 [31759,
  19218,
  60613,
  63339,
  4491,
  26898,
  39252,
  40724,
  60558,
  51549,
  56847,
  60613,
  47410,
  57101,
  63585,
  42279,
  4606,
  19132,
  23224,
  8836,
  60904,
  22551,
  41346],
 [53115,
  58271,
  42688,
  5383,
  60558,
  5

In [69]:
MAX_length = 50
padded_docs = pad_sequences(encoded_docs, maxlen=MAX_length, padding='post') #Make all sentences of the same length.
print(padded_docs)

[[49886 60558 17844 ...     0     0     0]
 [35387 48962 25945 ...     0     0     0]
 [33806 61119 46142 ...     0     0     0]
 ...
 [  959 43162 39746 ...     0     0     0]
 [15412 48962  3059 ...     0     0     0]
 [24111 57023 27560 ...     0     0     0]]


Now we build and train a model.

In [70]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=MAX_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           6361600   
_________________________________________________________________
flatten_1 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5001      
Total params: 6,366,601
Trainable params: 6,366,601
Non-trainable params: 0
_________________________________________________________________
None


In [73]:
model.fit(padded_docs, labels_train, epochs=100, verbose=0)

The training takes forever. So please load the model included in the directory.

In [71]:
from keras.models import load_model
model = load_model('task5.model')

Instructions for updating:
Use tf.cast instead.


# 6. Оценка результатов бинарной классификации

The next step is to make the files for testing.

In [73]:
test = pd.read_csv('test.csv',sep = '\t', quoting=csv.QUOTE_NONE,encoding='utf-8').fillna(0)

In [74]:
docs_test, labels_test, text_index_test = clear_text(test, 50)
encoded_test = [one_hot(d, vocab_size) for d in docs_test]
padded_test = pad_sequences(encoded_test, maxlen=MAX_length, padding='post')

In [75]:
loss, accuracy = model.evaluate(padded_test, labels_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 59.177533


The achieved accuracy is 59.2%.

! python3 agrr_metrics.py -b test.csv output.csv

# 7. Классификация последовательности

In [78]:
train_encoded = IOB_encoder(train)

In [80]:
to_filter = [train_encoded[i] for i in range(len(train))if i not in text_index]#Filter from train_encoded those sentences that are too long.
for sent in to_filter:
    train_encoded.remove(sent)

In [82]:
train_encoded

[{'Будучи': 'O',
  'в': 'O',
  'прошлый': 'O',
  'четверг': 'O',
  'Софии,': 'O',
  'он': 'O',
  'назвал': 'O',
  'себя': 'O',
  "'наполовину": 'O',
  "восточноевропейцем',": 'O',
  'и': 'O',
  'за': 'O',
  'последние': 'O',
  'пять': 'O',
  'дней': 'O',
  'успел': 'O',
  'принять': 'O',
  'у': 'O',
  'лидеров': 'O',
  'Украины,': 'O',
  'Польши': 'O',
  'Чехии': 'O'},
 {'состоит': 'B-cV',
  'часть их': 'B-cR1',
  'одного': 'B-cR2',
  '': 'V',
  'часть': 'B-R1',
  'из': 'B-R2',
  'двух': 'I-R2',
  'Работа': 'O',
  'с': 'O',
  'двухбайтовыми': 'O',
  'наборами': 'O',
  'символов': 'O',
  '—': 'O',
  'просто': 'O',
  'кошмар': 'O',
  'для': 'O',
  'программиста,': 'O',
  'так': 'O',
  'как': 'O',
  'байта,': 'O',
  'а': 'O',
  '.': 'O'},
 {'Заместитель': 'O',
  'Генерального': 'O',
  'секретаря': 'O',
  'подчеркнул,': 'O',
  'что': 'O',
  'проблема': 'O',
  'наркотиков': 'O',
  'не': 'O',
  'только': 'O',
  'дестабилизирует': 'O',
  'обстановку': 'O',
  'в': 'O',
  'стране,': 'O',
  'но'

In [87]:
def process_encoded(encoded):
    sentences = []
    IOBs = []
    
    ru = ' йцукенгшщзхъфывапролджэячсмитьбюё'
    for sent in encoded:
        words_list = []
        IOB_list = []
        for phrase in sent:
            IOB_current = sent[phrase]
            phrase = ''.join([letter for letter in phrase if letter in ru])
            words = word_tokenize(phrase)
            leng = words
            for word in words:
                words_list.append(word)
                IOB_list.append(IOB_current)
            
        sentences.append(words_list)
        IOBs.append(IOB_list)
    
    
    
    return sentences, IOBs

train_sent, train_IOB = process_encoded(train_encoded)

In [94]:
def make_IOB_index(encoded):
    '''
    Create a vocabulary for label indexing.
    '''
    IOB_index = {}
    n = 0
    for i in encoded:
        for j in i.values():
            if j not in IOB_index:
                IOB_index[j] = n
                n += 1
    
    return IOB_index

IOB_index = make_IOB_index(train_encoded)

In [183]:
import numpy as np
def labels_IOB(IOB, MAX_length, IOB_index):
    '''
    Create a list of lists representing the IOB properties of each word.
    '''
    labels = []
    for i in range(len(IOB)):
        add = [0]*MAX_length
        for j in range(len(IOB[i])):
            add[j] = IOB_index[IOB[i][j]]
        labels.append(add)
        
    return labels

IOB_train_labels = labels_IOB(train_IOB, 50, IOB_index)

Now we build the model.

In [176]:
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam

model = Sequential()
model.add(InputLayer(input_shape=(MAX_length, )))
model.add(Embedding(vocab_size, 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(IOB_index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
 
model.summary()
 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 50, 128)           8142848   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 50, 512)           788480    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 50, 9)             4617      
_________________________________________________________________
activation_2 (Activation)    (None, 50, 9)             0         
Total params: 8,935,945
Trainable params: 8,935,945
Non-trainable params: 0
_________________________________________________________________


In [185]:
def to_categorical(sequences, categories):
    '''
    Turn the index-encoding for labels into categorical-encoding.
    '''
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [258]:
cat_train_tags_y = to_categorical(IOB_train_labels, len(IOB_index))

In [189]:
model.fit(padded_docs, cat_train_tags_y, epochs=50, batch_size=128, validation_split=0.2)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 12716 samples, validate on 3180 samples
Epoch 1/50
12716/12716 [==============================] - 60s 5ms/step - loss: 0.3972 - acc: 0.9200 - val_loss: 0.2671 - val_acc: 0.9267
Epoch 2/50
12716/12716 [==============================] - 58s 5ms/step - loss: 0.2235 - acc: 0.9311 - val_loss: 0.2483 - val_acc: 0.9243
Epoch 3/50
12716/12716 [==============================] - 59s 5ms/step - loss: 0.1547 - acc: 0.9461 - val_loss: 0.2668 - val_acc: 0.9307
Epoch 4/50
12716/12716 [==============================] - 59s 5ms/step - loss: 0.1220 - acc: 0.9573 - val_loss: 0.2838 - val_acc: 0.9321
Epoch 5/50
12716/12716 [==============================] - 60s 5ms/step - loss: 0.1042 - acc: 0.9640 - val_loss: 0.3289 - val_acc: 0.9275
Epoch 6/50
12716/12716 [==============================] - 60s 5ms/step - loss: 0.0921 - acc: 0.9676 - val_loss: 0.3537 - val_acc: 0.9286
Epoch 7/50
12716/12716 [===========================

In [195]:
model.save('IOB.model')

In [ ]:
model = load_model('IOB.model')

# 8. Оценка результатов полной разметки

In [196]:
test_encoded = IOB_encoder(test)

In [197]:
to_filter = [test_encoded[i] for i in range(len(test))if i not in text_index_test]
for sent in to_filter:
    test_encoded.remove(sent)

In [198]:
test_sent, test_IOB = process_encoded(test_encoded)

In [199]:
IOB_test_labels = labels_IOB(test_IOB, 50, IOB_index)

In [194]:
cat_test_tags_y = to_categorical(IOB_test_labels, len(IOB_index))

In [201]:
scores = model.evaluate(padded_test, cat_test_tags_y)
print(f"{model.metrics_names[1]}: {scores[1] * 100}")

1994/1994 [==============================] - 3s 1ms/step
acc: 92.1985959337133


The achieved accuracy is 92.2%. The next step is to make a prediction.

In [224]:
predictions = model.predict(padded_test)

In [227]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [229]:
pred_results = logits_to_tokens(predictions, {i: t for t, i in IOB_index.items()})

In [262]:
index = 1

In [270]:
print('The original word bag:', test_sent[index])
print('The original IOB properties:', test_IOB[index])
print('The predicted IOB:', pred_results[index])


The original word bag: ['называют', 'ти', 'состояния', 'фазами', 'воды', 'превращения', 'из', 'одного', 'состояния', 'в', 'другое', 'фазовыми', 'переходами', 'а']
The original IOB properties: ['B-cV', 'B-cR1', 'B-cR1', 'B-cR2', 'I-cR2', 'B-R1', 'B-R1', 'B-R1', 'B-R1', 'B-R1', 'B-R1', 'B-R2', 'I-R2', 'O']
The predicted IOB: ['B-cV', 'B-cR1', 'B-cR1', 'I-cR2', 'B-R1', 'B-R1', 'B-R1', 'I-R2', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


We can see that the sequence of prediction is fairly similar. If one word is predicted wrong, the chain effect will probably make the predictions for the following words wrong.